# Data Wrangling: Join, Combine, and Reshape
* This chapter focuses on tools to help combine, join, and rearrange data



## Hierarchical Indexing

In [7]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.uniform(size=9),index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],[1, 2, 3, 1, 3, 1, 2, 2, 3]])
data


a  1    0.142332
   2    0.914541
   3    0.757539
b  1    0.535165
   3    0.785754
c  1    0.422658
   2    0.186120
d  2    0.898947
   3    0.881008
dtype: float64

In [8]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [9]:
data["b"]

1    0.535165
3    0.785754
dtype: float64

In [10]:
data.loc[:,2]

a    0.914541
c    0.186120
d    0.898947
dtype: float64

In [11]:
data.unstack()

,1,2,3
a,0.142332,0.914541,0.757539
b,0.535165,NaN,0.785754
c,0.422658,0.186120,NaN
d,NaN,0.898947,0.881008


In [12]:
data.unstack().stack()

a  1    0.142332
   2    0.914541
   3    0.757539
b  1    0.535165
   3    0.785754
c  1    0.422658
   2    0.186120
d  2    0.898947
   3    0.881008
dtype: float64

In [13]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index = [["a","a","b","b"],[1,2,1,2]],
                    columns=[["Ohio", "Ohio", "Colorado"],
                             ["Green","Red","Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [16]:
frame.index.names = ["key1","key2"]
frame.columns.names = ["state","color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
["Green", "Red", "Green"]],
names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

In [18]:
frame.swaplevel("key1","key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [19]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [20]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
....: "c": ["one", "one", "one", "two", "two",
....: "two", "two"],
....: "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [21]:
frame2 = frame.set_index(["c","d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

## Combining and Merging Datasets

In [22]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
....: "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
....: "data2": pd.Series(range(3), dtype="Int64")})

In [23]:
#Merge or join operations combine datasets by linking rows using one or more keys
pd.merge(df1,df2)


,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [25]:
pd.merge(df1,df2,on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [27]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
....: "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
....: "data2": pd.Series(range(3), dtype="Int64")})
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [29]:
# The outer join takes the union of the keys,
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [30]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [33]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
....: "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [34]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [35]:
left1.join(right1, how="outer")

,key,value,group_val
0,a,0,NaN
1,b,1,NaN
2,a,2,NaN
3,a,3,NaN
4,b,4,NaN
5,c,5,NaN
a,NaN,<NA>,3.5
b,NaN,<NA>,7.0


In [36]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
....: index=["a", "c", "e", "f"],
....: columns=["New York", "Oregon"])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [37]:
left1.join([right1, another])

,key,value,group_val,New York,Oregon
0,a,0,NaN,NaN,NaN
1,b,1,NaN,NaN,NaN
2,a,2,NaN,NaN,NaN
3,a,3,NaN,NaN,NaN
4,b,4,NaN,NaN,NaN
5,c,5,NaN,NaN,NaN


In [38]:
# Sample DataFrame in wide format
df = pd.DataFrame({
    "key": ["foo", "bar", "baz"],
    "A": [1, 2, 3],
    "B": [4, 5, 6],
    "C": [7, 8, 9]
})

In [39]:
# Melting to long format
melted = pd.melt(df, id_vars="key", value_vars=["A", "B", "C"])
print("Melted DataFrame:\n", melted)

Melted DataFrame:
    key variable  value
0  foo        A      1
1  bar        A      2
2  baz        A      3
3  foo        B      4
4  bar        B      5
5  baz        B      6
6  foo        C      7
7  bar        C      8
8  baz        C      9


In [40]:
# Pivoting back to wide format
reshaped = melted.pivot(index="key", columns="variable", values="value")
reshaped_reset = reshaped.reset_index()
print("\nReshaped DataFrame:\n", reshaped_reset)


Reshaped DataFrame:
 variable  key  A  B  C
0         bar  2  5  8
1         baz  3  6  9
2         foo  1  4  7


In [43]:
# Sample DataFrame with a MultiIndex
data = pd.DataFrame({
    "left": [0, 1, 2, 3, 4, 5],
    "right": [5, 6, 7, 8, 9, 10]
}, index=pd.MultiIndex.from_product([["Ohio", "Colorado"], ["one", "two", "three"]],
                                    names=["state", "number"]))

print("Original DataFrame:\n", data)

# Unstack the 'state' level
unstacked = data.unstack(level="state")
print("\nUnstacked DataFrame:\n", unstacked)

# Stack it back
restacked = unstacked.stack(level="state")
print("\nRestacked DataFrame:\n", restacked)

Original DataFrame:
                  left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

Unstacked DataFrame:
            left         right     
state  Colorado Ohio Colorado Ohio
number                            
one           3    0        8    5
three         5    2       10    7
two           4    1        9    6

Restacked DataFrame:
                  left  right
number state                
one    Colorado     3      8
       Ohio         0      5
three  Colorado     5     10
       Ohio         2      7
two    Colorado     4      9
       Ohio         1      6


/tmp/ipykernel_1767/3160001534.py:15: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  restacked = unstacked.stack(level="state")


In [44]:
# Sample DataFrame in "long" format
data_long = pd.DataFrame({
    "date": ["2021-01-01", "2021-01-01", "2021-01-01", 
             "2021-02-01", "2021-02-01", "2021-02-01"],
    "item": ["realgdp", "infl", "unemp", "realgdp", "infl", "unemp"],
    "value": [2710.349, 0.00, 5.8, 2778.801, 2.34, 5.1]
})

print("Long DataFrame:\n", data_long)

# Pivot to "wide" format
pivoted = data_long.pivot(index="date", columns="item", values="value")
print("\nPivoted DataFrame:\n", pivoted)

Long DataFrame:
          date     item     value
0  2021-01-01  realgdp  2710.349
1  2021-01-01     infl     0.000
2  2021-01-01    unemp     5.800
3  2021-02-01  realgdp  2778.801
4  2021-02-01     infl     2.340
5  2021-02-01    unemp     5.100

Pivoted DataFrame:
 item        infl   realgdp  unemp
date                             
2021-01-01  0.00  2710.349    5.8
2021-02-01  2.34  2778.801    5.1


In [45]:
# Sample DataFrame in "wide" format
df = pd.DataFrame({
    "key": ["foo", "bar", "baz"],
    "A": [1, 2, 3],
    "B": [4, 5, 6],
    "C": [7, 8, 9]
})

print("Wide DataFrame:\n", df)

# Melt to "long" format
melted = pd.melt(df, id_vars="key", value_vars=["A", "B", "C"])
print("\nMelted DataFrame:\n", melted)

# Pivot back to "wide" format
reshaped = melted.pivot(index="key", columns="variable", values="value").reset_index()
print("\nReshaped DataFrame:\n", reshaped)

Wide DataFrame:
    key  A  B  C
0  foo  1  4  7
1  bar  2  5  8
2  baz  3  6  9

Melted DataFrame:
    key variable  value
0  foo        A      1
1  bar        A      2
2  baz        A      3
3  foo        B      4
4  bar        B      5
5  baz        B      6
6  foo        C      7
7  bar        C      8
8  baz        C      9

Reshaped DataFrame:
 variable  key  A  B  C
0         bar  2  5  8
1         baz  3  6  9
2         foo  1  4  7


In [46]:
# Adding an additional value column
data_long["value2"] = np.random.randn(len(data_long))

print("Long DataFrame with Multiple Values:\n", data_long)

# Pivot with hierarchical columns
pivoted = data_long.pivot(index="date", columns="item")
print("\nPivoted DataFrame with Hierarchical Columns:\n", pivoted)

Long DataFrame with Multiple Values:
          date     item     value    value2
0  2021-01-01  realgdp  2710.349  1.022011
1  2021-01-01     infl     0.000 -0.674207
2  2021-01-01    unemp     5.800  0.608539
3  2021-02-01  realgdp  2778.801  0.914145
4  2021-02-01     infl     2.340  0.214140
5  2021-02-01    unemp     5.100  0.949902

Pivoted DataFrame with Hierarchical Columns:
            value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
2021-01-01  0.00  2710.349   5.8 -0.674207  1.022011  0.608539
2021-02-01  2.34  2778.801   5.1  0.214140  0.914145  0.949902
